# Домашнее задание 3: Perception

В этом задании вам будет необходимо обучить PointNet для задачи фильтрации шума в лидарном облаке.

В 3 семинаре мы с вами придумавали руками фичи и пытались обучить на этих данных catboost. Практика показывает, что сетки куда более способные генераторы фичей.

[Данные](https://yadi.sk/d/CBoVCVIxJ2q2cw)

Задание:

1. Необходимо реализовать PointNet, который будет работать на данных со снегом из 3 семинара. PointNet должен работать на окрестностях точек, нет смысла запускать его на всем облаке. PointNet должен включать в себя шаг агрегации по множеству: например с помощью функции максимума, шаг подклеивания агрегированного вектора к исходным точкам и шаг вычисления фичей по отдельным точкам. Вероятно вы захотите повторить эту процедуру несколько раз для улучшения качества. Статья: https://arxiv.org/abs/1612.00593. Вы можете выбрать любой фреймворк для реализации.
2. Ваш PointNet должен ограничить сверху размер окрестности. В референсной реализации использовались 64 точки.
3. Разбиение на train/test. Для разбиения используйте следующий код.
```
scene_indices = np.arange(0, 291)
np.random.seed(100)
np.random.shuffle(scene_indices)
train_indices = scene_indices[:260]
test_indices = scene_indices[260:]
```
4. Данные лучше генерировать on-demand, таким образом вам не придется хранить в памяти большие массивы точек. В tensorflow это можно реализовать через tf.data.

5. PointNet это функция, которая работает на неупорядоченном множестве точек. В нашем же кейсе мы не хотим предсказать свойство окрестности, мы хотим предсказать свойство точки. Подумайте о том как можно модифицировать архитектуру, чтобы pointnet "не забывал" фичи точки, которая нам интересна. (Это поможет улучшить качество)


## Формальные требования

1. В вашей архитектуре должны быть признаки PointNet: вычисление глобального вектора множества, подклеивание его обратно, вычисление фичей по точкам.

2. ROC-AUC на тестовом датасете должен превышать 0.99


In [12]:
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader

import typing as tp

In [13]:
features = pd.read_csv('data/snow_features.csv', index_col=0)

/home/mikita-zhuryk/shad/shad-sdc/env/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [14]:
features.head(10)

,scene_id,x,y,z,intensity,ring,label,min_intensity_1.0,max_intensity_1.0,median_intensity_1.0,std_intensity_1.0,min_ring_1.0,max_ring_1.0,median_ring_1.0,std_ring_1.0,r_std_1.0,n_neighbours_1.0
0,0.0,-11.355618,-4.206962,0.344085,0.0,23.0,1.0,0.0,0.0,0.0,0.000000,23.0,23.0,23.0,0.000000,0.000000,1.0
1,0.0,-5.916535,-1.972164,0.283262,0.0,25.0,1.0,0.0,0.0,0.0,0.000000,25.0,25.0,25.0,0.000000,0.000000,1.0
2,0.0,-7.410451,-2.113039,2.137792,0.0,31.0,1.0,0.0,0.0,0.0,0.000000,31.0,31.0,31.0,0.000000,0.000000,1.0
3,0.0,-13.845870,-1.406652,0.406310,0.0,23.0,1.0,0.0,0.0,0.0,0.000000,23.0,23.0,23.0,0.000000,0.000000,1.0
4,0.0,-8.326218,-0.346060,0.226469,0.0,22.0,1.0,0.0,0.0,0.0,0.000000,22.0,22.0,22.0,0.000000,0.000000,1.0
5,0.0,-29.016968,-2.179385,0.945424,7.0,24.0,1.0,7.0,7.0,7.0,0.000000,24.0,24.0,24.0,0.000000,0.000000,1.0
6,0.0,-2.074985,0.003017,0.044024,2.0,16.0,1.0,2.0,21.0,3.0,8.730534,16.0,27.0,17.0,4.966555,0.192132,3.0
7,0.0,-2.041912,-0.009894,0.055311,3.0,17.0,1.0,2.0,21.0,3.0,8.730534,16.0,27.0,17.0,4.966555,0.189939,3.0
8,0.0,-6.275961,0.790447,0.086301,0.0,19.0,1.0,0.0,0.0,0.0,0.000000,19.0,25.0,22.0,3.000000,0.041361,2.0
9,0.0,-8.290426,1.923754,0.044705,0.0,18.0,1.0,0.0,0.0,0.0,0.000000,18.0,23.0,20.5,2.500000,0.028832,2.0


In [67]:
from sklearn.neighbors import KDTree

class PointCloudDataset(Dataset):
    def __init__(self, data_df: pd.DataFrame) -> None:
        self.df: pd.DataFrame = data_df.reset_index(drop=True)
        self.scene_ids = self.df.scene_id.unique().tolist()
        self.n_scenes = len(self.scene_ids)
        
    def __getitem__(self, scene_idx: int) -> tp.Any:
        return self.df[self.df.scene_id == self.scene_ids[scene_idx]]
    
    def __len__(self) -> int:
        return self.n_scenes
    
    
class SceneDataset(Dataset):
    def __init__(self, cloud_df: pd.DataFrame) -> None:
        self.df: pd.DataFrame = cloud_df.reset_index(drop=True)
        self.tree = KDTree(self.df[['x', 'y', 'z']].to_numpy())
        self.labels = self.df.label
        self.features = self.df.drop(columns=['label', 'scene_id'])
        
    def __getitem__(self, idx: int) -> tp.Any:
        point = self.features.iloc[idx, :3].to_numpy()
        neighbor_ids, _ = self.tree.query_radius(point[np.newaxis, ...], r=1,
                                                 return_distance=True, sort_results=True)
        neighbor_ids = neighbor_ids[0]
        return self.features.iloc[neighbor_ids].to_numpy(), self.labels.iloc[idx]
    
    def __len__(self) -> int:
        return self.df.shape[0]

In [68]:
scene_indices = np.arange(0, 291)
np.random.seed(100)
np.random.shuffle(scene_indices)
train_indices = scene_indices[:260]
test_indices = scene_indices[260:]

train_data = features[features.scene_id.isin(train_indices)]
test_data = features[features.scene_id.isin(test_indices)]

In [ ]:
train_data = PointCloudDataset(train_data)
test_data = PointCloudDataset(test_data)

In [ ]:
import typing as tp

import numpy as np
import torch

from sklearn import metrics as M

NAME_TO_METRIC = {'accuracy': M.accuracy_score,
                  'recall': M.recall_score,
                  'precision': M.precision_score,
                  'f1': M.f1_score,
                  'roc_auc': M.roc_auc_score}


class ClassificationMetricLogger:
    def __init__(self, n_classes: int, metrics: tp.List[str] = ['precision', 'recall', 'f1']) -> None:
        self.n_metrics = len(metrics)
        self.metrics = metrics
        self.train_losses: tp.List[float] = []
        self.train_preds: tp.List[int] = []
        self.train_gt: tp.List[int] = []
        self.val_losses: tp.List[float] = []
        self.val_preds: tp.List[int] = []
        self.val_gt: tp.List[int] = []
        self._train = True
        self.n_classes = n_classes

    def train(self, train: bool = True) -> None:
        self._train = train

    def eval(self) -> None:
        self._train = False

    def __logits_to_classes(self, logits: torch.Tensor) -> tp.List[int]:
        return tp.cast(tp.List[int], torch.argmax(logits, dim=1).numpy().astype(int).tolist())

    def process_predictions(self, preds: torch.Tensor, gt: torch.Tensor, loss: float) -> None:
        classes = self.__logits_to_classes(preds)
        gt = gt.numpy().tolist()
        if self._train:
            self.train_losses.append(loss)
            self.train_preds.extend(classes)
            self.train_gt.extend(gt)
        else:
            self.val_losses.append(loss)
            self.val_preds.extend(classes)
            self.val_gt.extend(gt)

    def __metrics(self, train: bool = False) -> tp.Dict[str, float]:
        if train:
            losses = self.train_losses
            preds = self.train_preds
            gt = self.train_gt
        else:
            losses = self.val_losses
            preds = self.val_preds
            gt = self.val_gt

        metric_dict = {'mean_loss': float(np.mean(losses))}
        for metric in self.metrics:
            metric_dict[metric] = float(NAME_TO_METRIC[metric](gt, preds,
                                                               labels=np.arange(self.n_classes),
                                                               average="weighted"))
        return metric_dict

    def __describe_split(self, train: bool = True) -> str:
        m = self.__metrics(train)
        s = ''
        for (k, v) in m.items():
            s += f'{k}: {v}\n'
        return s

    def train_metrics(self) -> tp.Dict[str, float]:
        return self.__metrics(train=True)

    def val_metrics(self) -> tp.Dict[str, float]:
        return self.__metrics(train=False)

    def get_summary(self) -> str:
        s = 'Train metrics:\n'
        s += self.__describe_split(train=True)
        s += 'Val metrics:\n'
        s += self.__describe_split(train=False)
        return s

    def print_summary(self) -> None:
        print(self.get_summary())

    def reset(self) -> None:
        self.train_losses = []
        self.train_preds = []
        self.train_gt = []
        self.val_losses = []
        self.val_preds = []
        self.val_gt = []

In [ ]:
# DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE = 'cpu'
N_EPOCHS = 45

In [ ]:
# def compute_loss(pred: torch.Tensor, gt: torch.Tensor) -> torch.Tensor:
#     return torch.nn.functional.binary_cross_entropy_with_logits(pred, gt)

compute_loss = torch.nn.CrossEntropyLoss()

In [ ]:
from tqdm.notebook import tqdm

def single_epoch(model: torch.nn.Module,
                 optimizer: torch.optim.Adam,
                 train_data: PointCloudDataset,
                 test_data: PointCloudDataset,
                 compute_loss: tp.Callable[[torch.Tensor, torch.Tensor], torch.Tensor],
                 metric_logger: ClassificationMetricLogger) -> None:
    model.train()
    metric_logger.train()
    for scene_df in tqdm(train_data, desc='Train scenes'):
        scene_data = SceneDataset(scene_df)
        scene_loader = DataLoader(scene_data, batch_size=1, num_workers=1, shuffle=True)
        for (features, gt) in scene_loader:
            features = features.float()
            gt = gt.long()
            optimizer.zero_grad()
            pred = model(features.to(DEVICE))
            print(pred.shape, gt.shape)
            loss = compute_loss(pred, gt.to(DEVICE))
            loss.backward()
            optimizer.step()
            metric_logger.process_predictions(pred.detach().cpu(), gt, loss.detach().cpu().item())
            
    model.eval()
    metric_logger.eval()
    with torch.no_grad():
        for scene_df in tqdm(test_data, desc='Test scenes'):
            scene_data = SceneDataset(scene_df)
            scene_loader = DataLoader(scene_data, batch_size=1, num_workers=1, shuffle=True)
            for (features, gt) in scene_loader:
                features = features.float()
                gt = gt.long()
                pred = model(features.to(DEVICE))
                loss = compute_loss(pred, gt.to(DEVICE))
                metric_logger.process_predictions(pred.cpu(), gt, loss.cpu().item())

In [ ]:
from torch import nn

class PointNetModel(nn.Module):
    def __init__(self, in_features: int = 15, n_out_classes: int = 2) -> None:
        super().__init__()
        self.mlp = nn.Sequential(nn.Linear(in_features, 64),
                                 nn.Linear(64, 64))
        self.embedding_mlp = nn.Sequential(nn.Linear(64, 128),
                                           nn.Linear(128, 1024))
        clf_mlp_features = 64 + 1024
        get_clf_mlp = lambda: nn.Linear(clf_mlp_features, 1024)
        self.combining_mlps = torch.nn.ModuleList([get_clf_mlp() for _ in range(3)])
        self.clf_mlp = nn.Sequential(nn.Linear(clf_mlp_features + in_features, 512),
                                     nn.Linear(512, 128),
                                     nn.Linear(128, n_out_classes))
        
        
    def forward(self, x):
        inputs = x
        x64 = self.mlp(x)
        global_features = torch.max(self.embedding_mlp(x64), dim=1).values.view(1, 1, -1)
        x = torch.cat((x64, torch.tile(global_features, (1, inputs.shape[1], 1))), dim=2)
        for i in range(len(self.combining_mlps)):
            global_features = torch.max(self.combining_mlps[i](x), dim=1).values.view(1, 1, -1)
            x = torch.cat((x64, torch.tile(global_features, (1, inputs.shape[1], 1))), dim=2)
        x = torch.cat((inputs, x), dim=2)
        prediction = self.clf_mlp(x)[:, 0]
        return torch.squeeze(prediction, dim=-1)

In [ ]:
model = PointNetModel()
model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.9)
metric_logger = ClassificationMetricLogger(n_classes=2,
                                           metrics=['precision', 'recall', 'f1', 'roc_auc'])
start_epoch = 0

In [ ]:
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ONL9sI2e_HG7pQ9zYVM-JSVqzsR_8Pnn' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1ONL9sI2e_HG7pQ9zYVM-JSVqzsR_8Pnn" -O pointnet-4.pth && rm -rf /tmp/cookies.txt

# state_dict = torch.load('pointnet-4.pth', map_location='cpu')
# model.cpu()
# model.load_state_dict(state_dict['model'])
# model.to(DEVICE)
# optimizer.load_state_dict(state_dict['optimizer'])
# scheduler.load_state_dict(state_dict['scheduler'])
# start_epoch = state_dict['epoch']

In [ ]:
from tqdm.notebook import tqdm

for ep in tqdm(range(start_epoch, N_EPOCHS), desc='Epoch'):
    single_epoch(model, optimizer, train_data, test_data, compute_loss, metric_logger)
    print(f'Epoch {ep}:')
    metric_logger.print_summary()
    scheduler.step()
    state_dict = {'model': model.state_dict(),
                  'optimizer': optimizer.state_dict(),
                  'scheduler': scheduler.state_dict(),
                  'epoch': ep}
    torch.save(state_dict, f'pointnet-{ep}.pth')
    metric_logger.reset()